In [8]:
def gerar_coordenadas_normais(dimensao_maxima, dimensoes_caixas):
    """
    Gera um conjunto de coordenadas otimizado ('normal patterns') para um único eixo.

    Args:
        dimensao_maxima (int): O tamanho do contêiner nesse eixo (L, W ou H).
        dimensoes_caixas (list or set): Uma coleção com as dimensões únicas
                                         das caixas para esse eixo (ex: todos os comprimentos).

    Returns:
        list: Uma lista ordenada de coordenadas permitidas.
    """
    
    # 1. Comece com o ponto de origem. Usamos um 'set' para evitar duplicatas.
    coordenadas = {0}

    # 2. Itere sobre cada dimensão de caixa única.
    for d in sorted(list(dimensoes_caixas)):
        # Criamos uma cópia para adicionar os novos pontos, para não modificar o
        # conjunto enquanto iteramos sobre ele.
        novas_coordenadas_a_adicionar = set()
        
        # Para cada coordenada que já encontramos...
        for c in coordenadas:
            # ...some múltiplos da dimensão atual.
            novo_ponto = c + d
            while novo_ponto <= dimensao_maxima:
                novas_coordenadas_a_adicionar.add(novo_ponto)
                novo_ponto += d
        
        # Adiciona todos os novos pontos encontrados ao nosso conjunto principal.
        coordenadas.update(novas_coordenadas_a_adicionar)

    # Uma pequena otimização final: remove pontos onde nem a menor caixa cabe.
    if not dimensoes_caixas: return [0] # Evita erro se a lista for vazia
    menor_dimensao = min(dimensoes_caixas)
    coordenadas_finais = [c for c in coordenadas if c <= dimensao_maxima - menor_dimensao]
    
    # Adiciona a coordenada 0 se ela não estiver na lista (caso especial)
    if 0 not in coordenadas_finais:
        coordenadas_finais.insert(0,0)

    return sorted(coordenadas_finais)

In [9]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product

def escrever_solucao_grid(x, box_types, L, W, H, filename):
    """
    Função adaptada para escrever a solução do modelo de grade em um arquivo.
    """
    print(f"\n--- Escrevendo solução no arquivo '{filename}'... ---")
    try:
        with open(filename, 'w') as f:
            # Escreve a primeira linha com as dimensões do contêiner
            f.write(f"{L} {W} {H}\n")
            
            # Itera sobre as variáveis de decisão para encontrar as caixas empacotadas
            for i, p, q, r in x:
                if x[i, p, q, r].X > 0.5:
                    li, wi, hi = box_types[i]['dims']
                    # O 'tipo' é o próprio índice 'i' do tipo de caixa (adicionamos 1 para ser mais legível)
                    tipo = i + 1
                    cliente = 1
                    
                    # Escreve a linha formatada para a caixa
                    f.write(f"{p} {q} {r} {li} {wi} {hi} {tipo} {cliente}\n")
        
        print("Arquivo de solução salvo com sucesso.")
    except Exception as e:
        print(f"ERRO: Não foi possível escrever no arquivo de solução. Causa: {e}")


def executar_e_relatar_grid(model, x, box_types, L, W, H, time_limit_seconds, output_filename):
    """
    Função adaptada para executar o modelo de grade, relatar os resultados 
    e salvar a solução.
    """
    model.setParam('TimeLimit', time_limit_seconds)
    model.setParam('OutputFlag', 0)

    print(f"--- Executando modelo de grade com limite de tempo de {time_limit_seconds} segundos... ---")
    
    model.optimize()

    print("\n" + "="*45)
    print("      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)")
    print("="*45)

    if model.SolCount > 0:
        status_solucao = "Ótima" if model.status == GRB.OPTIMAL else "Subótima (Limite de Tempo Atingido)"
        
        print(f"Status da Solução: {status_solucao}")
        print(f"Valor Final Atingido (FO): {model.ObjVal:.4f}")
        print(f"Gap de Otimalidade: {model.MIPGap * 100:.4f}%")
        print(f"Tempo Computacional Despendido: {model.Runtime:.4f} segundos")
        
        # Chama a função adaptada para escrever a solução em um arquivo
        escrever_solucao_grid(x, box_types, L, W, H, output_filename)
    else:
        print("Status da Solução: Nenhuma solução viável foi encontrada.")
        print(f"Tempo Computacional Despendido: {model.Runtime:.4f} segundos")

    print("="*45 + "\n")


def solve_grid_based_model(L, W, H, box_types, filename):
    """
    Implementa e resolve o modelo de grade, usando as funções de relatório.
    """
    try:
        # --- 1. Dados do Modelo ---
        m = len(box_types)

        all_lengths = {bt['dims'][0] for bt in box_types.values()}
        all_widths = {bt['dims'][1] for bt in box_types.values()}
        all_heights = {bt['dims'][2] for bt in box_types.values()}
        X_coords = gerar_coordenadas_normais(L, all_lengths)
        Y_coords = gerar_coordenadas_normais(W, all_widths)
        Z_coords = gerar_coordenadas_normais(H, all_heights)

        # --- 3. Construção do Modelo ---
        model = gp.Model("GridBasedContainerLoading")
        
        x = {}
        for i in range(m):
            li, wi, hi = box_types[i]['dims']
            for p in [c for c in X_coords if c <= L - li]:
                for q in [c for c in Y_coords if c <= W - wi]:
                    for r in [c for c in Z_coords if c <= H - hi]:
                        x[i, p, q, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{p}_{q}_{r}")

        model.setObjective(gp.quicksum(box_types[i]['val'] * x[i, p, q, r] for i, p, q, r in x), GRB.MAXIMIZE)

        for s, t, u in product(X_coords, Y_coords, Z_coords):
            boxes_covering_stu = [x[i, p, q, r] for i, p, q, r in x if (p <= s < p + box_types[i]['dims'][0]) and (q <= t < q + box_types[i]['dims'][1]) and (r <= u < r + box_types[i]['dims'][2])]
            if boxes_covering_stu:
                model.addConstr(gp.quicksum(boxes_covering_stu) <= 1)

        for i in range(m):
            placements_of_type_i = gp.quicksum(x[i, p, q, r] for i_p, p, q, r in x if i_p == i)
            model.addConstr(placements_of_type_i <= box_types[i]['b'])

        # --- 4. Execução e Relatório ---
        executar_e_relatar_grid(
            model=model,
            x=x,
            box_types=box_types,
            L=L, W=W, H=H,
            time_limit_seconds=3600,
            output_filename=filename
        )

    except gp.GurobiError as e:
        print(f"Gurobi Error code {e.errno}: {e}")

# Instancia 1

In [10]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (63, 64, 39), 'b': 2, 'val': (63*64*39/(L*W*H))},
    1: {'dims': (30, 27, 29), 'b': 2, 'val': (30*27*29/(L*W*H))},
    2: {'dims': (53, 46, 58), 'b': 2, 'val': (53*46*58/(L*W*H))},
    3: {'dims': (62, 52, 42), 'b': 1, 'val': (62*52*42/(L*W*H))},
    4: {'dims': (58, 54, 73), 'b': 1, 'val': (58*54*73/(L*W*H))},
}
filename = 'gridbased_instancia_1.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.4870
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0215 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_1.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 2

In [11]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (43, 57, 70), 'b': 1, 'val': (43*57*70/(L*W*H))},
    1: {'dims': (59, 45, 62), 'b': 1, 'val': (59*45*62/(L*W*H))},
    2: {'dims': (57, 65, 46), 'b': 2, 'val': (57*65*46/(L*W*H))},
    3: {'dims': (27, 27, 62), 'b': 4, 'val': ((27*27*62)/(L*W*H))},
    4: {'dims': (38, 38, 39), 'b': 7, 'val': (38*38*39/(L*W*H))},
}
filename = 'gridbased_instancia_2.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7155
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0106 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_2.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 3

In [12]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (36, 52, 33), 'b': 5, 'val': (36*52*33/(L*W*H))},
    1: {'dims': (53, 27, 30), 'b': 4, 'val': ((53* 27* 30)/(L*W*H))},
    2: {'dims': (45, 29, 62), 'b': 3, 'val': ((45*29*62)/(L*W*H))},
    3: {'dims': (40, 60, 65), 'b': 2, 'val': ((40*60*65)/(L*W*H))},
    4: {'dims': ( 65, 51, 43), 'b': 2, 'val': (65*61*43/(L*W*H))},
}
filename = 'gridbased_instancia_3.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7073
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0148 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_3.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 4

In [13]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (55, 71, 68), 'b': 1, 'val': (55*71*68/(L*W*H))},
    1: {'dims': (62, 71, 51), 'b': 1, 'val': ((62*71*51)/(L*W*H))},
    2: {'dims': (31, 26, 66), 'b': 9, 'val': ((31*26*66)/(L*W*H))},
    3: {'dims': (28, 56, 66), 'b': 2, 'val': ((28*56*66)/(L*W*H))},
    4: {'dims': (61, 68, 50), 'b': 1, 'val': (61*68*50/(L*W*H))},
}
filename = 'gridbased_instancia_4.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.5315
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0020 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_4.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 5

In [15]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (67, 29, 40), 'b': 6, 'val': ((67* 29* 40)/(L*W*H))},
    1: {'dims': (70, 42, 36), 'b': 3, 'val': ((70* 42* 36)/(L*W*H))},
    2: {'dims': (64, 48, 68), 'b': 1, 'val': ((64* 48* 68)/(L*W*H))},
    3: {'dims': (37, 55, 56), 'b': 1, 'val': ((37*55*56)/(L*W*H))},
    4: {'dims': (30, 71, 45), 'b': 5, 'val': (30*71*45/(L*W*H))},
}
filename = 'gridbased_instancia_5.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7405
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0247 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_5.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 6

In [16]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (72, 25, 26), 'b': 7, 'val': ((72*25*26)/(L*W*H))},
    1: {'dims': (43, 38, 71), 'b': 1, 'val': ((43*38*71)/(L*W*H))},
    2: {'dims': (64, 65, 74), 'b': 1, 'val': ((64*65*74)/(L*W*H))},
    3: {'dims': (54, 61, 60), 'b': 1, 'val': ((54*61*60)/(L*W*H))},
    4: {'dims': (27, 55, 52), 'b': 1, 'val': (27*55*52/(L*W*H))},
}
filename = 'gridbased_instancia_6.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.6717
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0336 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_6.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 7

In [19]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (60, 40, 58), 'b': 2, 'val': ((60*40*58)/(L*W*H))},
    1: {'dims': (71, 39, 43), 'b': 3, 'val': ((71*39*43)/(L*W*H))},
    2: {'dims': (68, 69, 54), 'b': 1, 'val': ((68*69*54)/(L*W*H))},
    3: {'dims': (38, 53, 53), 'b': 1, 'val': ((38*53*53)/(L*W*H))},
    4: {'dims': (27, 72, 54), 'b': 2, 'val': (27*72*54/(L*W*H))},
}
filename = 'gridbased_instancia_7.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.6739
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0059 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_7.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 8

In [20]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (27, 50, 46), 'b': 6, 'val': ((27*50*46)/(L*W*H))},
    1: {'dims': (65, 62, 43), 'b': 1, 'val': ((65*62*43)/(L*W*H))},
    2: {'dims': (45, 74, 55), 'b': 1, 'val': ((45*74*55)/(L*W*H))},
    3: {'dims': (48, 60, 69), 'b': 1, 'val': ((48*60*60)/(L*W*H))},
    4: {'dims': (49, 51, 34), 'b': 3, 'val': (49*51*34/(L*W*H))},
}
filename = 'gridbased_instancia_8.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7290
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0052 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_8.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 9

In [21]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (36, 38, 59), 'b': 3, 'val': ((36*38*59)/(L*W*H))},
    1: {'dims': (30, 55, 25), 'b': 4, 'val': ((30*55*25)/(L*W*H))},
    2: {'dims': (69, 59, 69), 'b': 1, 'val': ((69*59*69)/(L*W*H))},
    3: {'dims': (72, 47, 28), 'b': 4, 'val': ((72*47*28)/(L*W*H))},
    4: {'dims': (36, 55, 45), 'b': 2, 'val': (36*55*45/(L*W*H))},
}
filename = 'gridbased_instancia_9.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.7143
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0378 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_9.txt'... ---
Arquivo de solução salvo com sucesso.



# Instancia 10

In [22]:
# Executa o script principal
L, W, H = 100, 100, 100
box_types = {
    0: {'dims': (46, 33, 73), 'b': 6, 'val': ((46*33*73)/(L*W*H))},
    1: {'dims': (63, 72, 68), 'b': 1, 'val': ((63*72*68)/(L*W*H))},
    2: {'dims': (41, 29, 26), 'b': 9, 'val': ((41*29*26)/(L*W*H))},
    3: {'dims': (61, 31, 30), 'b': 4, 'val': ((61*31*30)/(L*W*H))},
    4: {'dims': (27, 48, 55), 'b': 1, 'val': (27*48*55/(L*W*H))},
}
filename = 'gridbased_instancia_10.txt'
solve_grid_based_model(L, W, H, box_types, filename)

Set parameter TimeLimit to value 3600
--- Executando modelo de grade com limite de tempo de 3600 segundos... ---

      RELATÓRIO FINAL DE EXECUÇÃO (MODELO DE GRADE)
Status da Solução: Ótima
Valor Final Atingido (FO): 0.8504
Gap de Otimalidade: 0.0000%
Tempo Computacional Despendido: 0.0295 segundos

--- Escrevendo solução no arquivo 'gridbased_instancia_10.txt'... ---
Arquivo de solução salvo com sucesso.

